In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dependencies

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install -q -U watermark

     |████████████████████████████████| 3.4 MB 2.1 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 61 kB 469 kB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.2 which is incompatible.


In [2]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy       : 1.19.5
pandas      : 1.1.5
torch       : 1.10.0+cu111
transformers: 4.15.0



In [3]:
!nvidia-smi

Thu Dec 23 01:22:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [23]:
class_names = ['non_hatespeech', 'hatespeech', 'racism', 'sexism', 'islamophobia']

In [6]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 5)
model = model.to(device)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [17]:
from transformers import XLNetTokenizer, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [11]:
model.load_state_dict(torch.load('/content/drive/MyDrive/FYP/FYP/trainedModels/xlnet_model2(epoch2).bin'))

<All keys matched successfully>

In [12]:
model = model.to(device)

In [18]:
MAX_LEN=128


In [70]:
#cleaning data

import re
def clean_text(text):
  text = re.sub("RT",' ',text)
  text = re.sub("!{3,}",' ',text)
  text = re.sub(r"&#8217;", "'", text)
  text = re.sub(r"&[#*a-zA-z0-9]+;", '', text)
  text = re.sub("@[_*a-zA-Z0-9]+:*",' ',text)
  text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  #text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
  text = re.sub('\t+', '',  text)
  text = re.sub(" {2,}",' ',text)
  text = re.sub("^ ",'',text)
  text = text.lower()
  #text = re.sub(r" +", ' ', text)
  return text

In [71]:
def predict_tweet(tweet, actual):
    
    review_text = tweet

    encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoded_review['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_review['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,MAX_LEN).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)

    print("non hate:", probs[0])
    print("hate:", probs[1])
    print("racism:", probs[2])
    print("sexism:", probs[3])
    print("islamophobia:", probs[4])
    print()
    print(f' text: {review_text}')
    print(f'type  : {class_names[prediction]}')
    print(f'actual  : {class_names[actual]}')
    print()

In [72]:
data = pd.DataFrame(columns = ['class', 'tweet'])

In [73]:
data = data.append({'class': 4, 'tweet':'i hate muslim'}, ignore_index=True)
data = data.append({'class': 4, 'tweet':'i hate Muslims'}, ignore_index=True)
data = data.append({'class': 4, 'tweet':"Here's a video of muslim jihadis"}, ignore_index=True)
data = data.append({'class': 4, 'tweet':'if muzrats reckon muzrat countries are houses of peace and non muzrat countries are houses of war then every muzrat in non muzrat countries should be considered as soldiers or spies behind enemy lines '}, ignore_index=True)

data = data.append({'class': 2, 'tweet':"i refuse to become homosexual but sometimes i really fucking hate these african american hood rat nigger bitches ðŸ goat "}, ignore_index=True)
data = data.append({'class': 2, 'tweet':"the nigger should just cut to the chase and break into the lot to steal the car  "}, ignore_index=True)
data = data.append({'class': 2, 'tweet':"i have a racist harley every time i crank it it says run nigger nigger nigger old but good  "}, ignore_index=True)
data = data.append({'class': 2, 'tweet':"what the fuck is this nigger announcer on nhl is nothing sacred like a fucking blackout on my <number> inch "}, ignore_index=True)

data = data.append({'class': 3, 'tweet':"I hate women"}, ignore_index=True)
data = data.append({'class': 3, 'tweet':"We know why she was hired..."}, ignore_index=True)
data = data.append({'class': 3, 'tweet':"High positions better suit to men"}, ignore_index=True)

data = data.append({'class': 1, 'tweet':" he's gone always be a hoe"}, ignore_index=True)
data = data.append({'class': 1, 'tweet':'" faggot read my tweets after dat k" it wasnt even funny lol'}, ignore_index=True)
data = data.append({'class': 1, 'tweet':"' On my way to fuck yo bitch ' me as a 9 year old"}, ignore_index=True)

In [74]:
data['tweet'] = list(map(clean_text, data['tweet']))

In [75]:
for i in range(len(data)):
  predict_tweet(data.loc[i,'tweet'], data.loc[i,'class'])

non hate: 0.0001631592313060537
hate: 0.00792025402188301
racism: 0.014077136293053627
sexism: 0.0039751906879246235
islamophobia: 0.9738643169403076

 text: i hate muslim
type  : islamophobia
actual  : islamophobia

non hate: 0.0001414647267665714
hate: 0.019025050103664398
racism: 0.005632358603179455
sexism: 0.002263137837871909
islamophobia: 0.9729380011558533

 text: i hate muslims
type  : islamophobia
actual  : islamophobia

non hate: 0.0002775914908852428
hate: 0.20009474456310272
racism: 0.0006633789162151515
sexism: 0.0006301730172708631
islamophobia: 0.7983340620994568

 text: here's a video of muslim jihadis
type  : islamophobia
actual  : islamophobia

non hate: 0.00018645645468495786
hate: 0.004621421918272972
racism: 0.02057265304028988
sexism: 0.004627141635864973
islamophobia: 0.9699923396110535

 text: if muzrats reckon muzrat countries are houses of peace and non muzrat countries are houses of war then every muzrat in non muzrat countries should be considered as soldie